In [ ]:
import pandas as pd
from pandas.io import gbq
import datetime
from datetime import date

filen = input('please provide the file name you want to import')

# import the txt file downloaded from SC 

df = pd.read_csv(f'/home/markaw/heroes/netsuite_cashsales/sc_flatfiles/{filen}.txt', delimiter = "\t",decimal=",",
                 dtype={"settlement-id": str, 
                        "settlement-start-date": str, 
                        "settlement-end-date": str, 
                        "deposit-date": str,
                        "total-amount": str, 
                        "adjustment-id": str, 
                        "shipment-id": str, 
                        "merchant-order-item-id": str}).iloc[1:,:]


try: 
    df['datetime'] = pd.to_datetime(df['posted-date-time'], format='%d.%m.%Y %H:%M:%S %Z')
except:
    df['datetime'] = pd.to_datetime(df['posted-date-time'], format='%Y-%m-%d %H:%M:%S %Z')
    
df['tranDate'] = df['datetime'].apply(lambda x: datetime.datetime.strftime(x, "%d/%m/%Y"))
df['postingPeriodRef'] =df['datetime'].apply(lambda x: datetime.datetime.strftime(x, "%b %Y"))

df['amount'] = pd.to_numeric(df['amount'])

# Map heroes ID (BQ), subsidiaries (local csv) and locations (local_csv/user input)

sql_query = f"""
SELECT distinct sku, heroes_product_id FROM analytics_viz.f_amz_order_items
"""

heroesid = gbq.read_gbq(sql_query, project_id = "analytics-298917")
df_heroes = df.merge(heroesid[['heroes_product_id','sku']], left_on='sku', right_on='sku')

customers = pd.read_csv('/home/markaw/heroes/netsuite_cashsales/Customer.csv', encoding='iso-8859-1')
subsidiaries = pd.read_csv('/home/markaw/heroes/netsuite_cashsales//subsidiaries.csv')

while True:
    brand = input('please provide Brand')
    if brand not in subsidiaries['Brand'].tolist():
        print('Please check the spelling for that brand')
        continue
    else:
        break

while True:
    country = input('please provide Country')
    if country not in customers['Country'].tolist():
        print('That country seems incorrect, check spelling')
        continue
    else:
        break

brand_ticker = subsidiaries[subsidiaries['Brand'] == brand].iloc[0][0].upper().replace(" ","")[0:3]
location = 'Amazon FBA - ' + (subsidiaries[subsidiaries['Brand'] ==  brand].iloc[0][1]) +' - '+ (customers[customers['Country'] == country].iloc[0][4])

# Filter for cash sale rows, create netsuite cash sale template, and export it into csv

cs_filter = df_heroes[((df_heroes['transaction-type'] == 'Order')&
                   (df_heroes['amount-description'].isin(['Principal'])))].copy()
cs_dict = {
    'ExternalID': f"""CS{brand_ticker}{date.today().strftime("%d%m")}{customers[customers['Country'] == country].iloc[0][3]}""",
    'tranId':' ',
    'customerRef': customers[customers['Country'] == country].iloc[0][1],
    'tranDate': cs_filter['tranDate'],
    'postingPeriodRef': cs_filter['postingPeriodRef'],
    'exchangeRate': '1',
    'locationRef':location,
    'isTaxable': 'TRUE',
    'itemLine_itemRef':cs_filter['heroes_product_id'],
    'itemLine_quantity':cs_filter['quantity-purchased'],
    'itemLine_unitsRef':'Units',
    'itemLine_salesPrice':cs_filter['amount']/cs_filter['quantity-purchased'],
    'itemLine_priceLevelRef':'Custom',
    'Subsiduary':subsidiaries[subsidiaries['Brand'] == brand].iloc[0][3],
    'Expected Deposit date':(datetime.datetime.strptime(cs_filter['tranDate'].max(), "%d/%m/%Y") + datetime.timedelta(days=15)).strftime("%d/%m/%Y")
}

cs_template = pd.DataFrame(cs_dict)



cs_name = f"""/home/markaw/heroes/netsuite_cashsales/cashsale_templates/CS_{brand_ticker}{date.today().strftime("%d%m")}{customers[customers['Country'] == country].iloc[0][3]}"""

for i in range(len(cs_template['postingPeriodRef'].unique().tolist())):
    cs_split = cs_template[cs_template['postingPeriodRef'] == cs_template['postingPeriodRef'].unique().tolist()[i]]
    cs_mon = cs_split.iloc[0][4][0:3]
    
    if len(cs_split) < 5000:
        cs_split.to_csv(f'{cs_name}_{cs_mon}.csv', index=False)
    
    elif 5000 <= len(cs_split) < 10000:
        cs_split[0:5000].to_csv(f'{cs_name}_{cs_mon}1.csv', index=False)

        cs_split2 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split2['ExternalID'] = cs_split['ExternalID']+'b'
        cs_split2.to_csv(f'{cs_name}_{cs_mon}2.csv', index=False)

    elif 10000 <= len(cs_split) < 15000:
        cs_split[0:5000].to_csv(f'{cs_name}_{cs_mon}1.csv', index=False)

        cs_split2 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split2['ExternalID'] = cs_split['ExternalID']+'b'
        cs_split2.to_csv(f'{cs_name}_{cs_mon}2.csv', index=False)

        cs_split3 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split3['ExternalID'] = cs_split['ExternalID']+'c'
        cs_split3.to_csv(f'{cs_name}_{cs_mon}3.csv', index=False)

    elif 15000 <= len(cs_split) < 20000:
        cs_split[0:5000].to_csv(f'{cs_name}_{cs_mon}1.csv', index=False)

        cs_split2 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split2['ExternalID'] = cs_split['ExternalID']+'b'
        cs_split2.to_csv(f'{cs_name}_{cs_mon}2.csv', index=False)

        cs_split3 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split3['ExternalID'] = cs_split['ExternalID']+'c'
        cs_split3.to_csv(f'{cs_name}_{cs_mon}3.csv', index=False)

        cs_split4 = cs_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        cs_split4['ExternalID'] = cs_split['ExternalID']+'c'
        cs_split4.to_csv(f'{cs_name}_{cs_mon}4.csv', index=False)
    

total_cs = (cs_template['itemLine_salesPrice']*cs_template['itemLine_quantity']).sum()
print(f'{len(cs_template)} CS rows exported! The total sum amount is {total_cs}')

# Filter for ancilliary rows, and create netsuite ancilliary template

anc_filter = df_heroes[((df_heroes['transaction-type'] == 'other-transaction')&
                   (df_heroes['amount-description'].isin(['Principal',
                                                          'Shipping',
                                                          'GiftWrap',
                                                          'RestockingFee',
                                                          'Replacement',
                                                          'Discount',
                                                          'Cupon'])))|
                ((df_heroes['transaction-type'] == 'Refund')&
                    (df_heroes['amount-description'].isin(['Principal',
                                                          'Shipping',
                                                          'Refund',
                                                          'Discount refund'])))].copy()

anc_dict = {
    'ExternalID': f"""AN{brand_ticker}{date.today().strftime("%d%m")}{customers[customers['Country'] == country].iloc[0][3]}""",
    'tranId':' ',
    'customerRef': customers[customers['Country'] == country].iloc[0][1],
    'tranDate': anc_filter['tranDate'],
    'postingPeriodRef': anc_filter['postingPeriodRef'],
    'exchangeRate': '1',
    'locationRef':location,
    'isTaxable': 'TRUE',
    'itemLine_itemRef':anc_filter['heroes_product_id'],
    'itemLine_quantity':'1',
    'itemLine_unitsRef':'Units',
    'itemLine_salesPrice':anc_filter['amount'],
    'itemLine_priceLevelRef':'Custom',
    'Subsiduary':subsidiaries[subsidiaries['Brand'] == brand].iloc[0][3],
    'Expected Deposit date':(datetime.datetime.strptime(anc_filter['tranDate'].max(), "%d/%m/%Y") + datetime.timedelta(days=15)).strftime("%d/%m/%Y")
}

anc_template = pd.DataFrame(anc_dict)

anc_name = f"""/home/markaw/heroes/netsuite_cashsales/cashsale_templates/AN_{brand_ticker}{date.today().strftime("%d%m")}{customers[customers['Country'] == country].iloc[0][3]}"""

for i in range(len(anc_template['postingPeriodRef'].unique().tolist())):
    anc_split = anc_template[anc_template['postingPeriodRef'] == anc_template['postingPeriodRef'].unique().tolist()[i]]
    anc_mon = anc_split.iloc[0][4][0:3]

    if len(anc_split) < 5000:
        anc_split.to_csv(f'{anc_name}_{anc_mon}.csv', index=False)
    
    elif 5000 <= len(anc_split) < 10000:
        anc_split[0:5000].to_csv(f'{anc_name}_{anc_mon}1.csv', index=False)

        anc_split2 = anc_split[(anc_split.index >= 5001) & ((anc_split.index < 10000))].copy()
        anc_split2['ExternalID'] = anc_split['ExternalID']+'b'
        anc_split2.to_csv(f'{anc_name}_{anc_mon}2.csv', index=False)

    elif 10000 <= len(anc_split) < 15000:
        anc_split[0:5000].to_csv(f'{cs_name}_{anc_mon}1.csv', index=False)

        anc_split2 = anc_split[(anc_split.index >= 5001) & ((anc_split.index < 10000))].copy()
        anc_split2['ExternalID'] = anc_split['ExternalID']+'b'
        anc_split2.to_csv(f'{anc_name}_{anc_mon}2.csv', index=False)

        anc_split3 = anc_split[(canc_split.index >= 5001) & ((anc_split.index < 10000))].copy()
        anc_split3['ExternalID'] = anc_split['ExternalID']+'c'
        anc_split3.to_csv(f'{anc_name}_{anc_mon}3.csv', index=False)

    elif 15000 <= len(anc_split) < 20000:
        anc_split[0:5000].to_csv(f'{cs_name}_{anc_mon}1.csv', index=False)

        anc_split2 = anc_split[(anc_split.index >= 5001) & ((anc_split.index < 10000))].copy()
        anc_split2['ExternalID'] = anc_split['ExternalID']+'b'
        anc_split2.to_csv(f'{anc_name}_{anc_mon}2.csv', index=False)

        anc_split3 = anc_split[(anc_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        anc_split3['ExternalID'] = anc_split['ExternalID']+'c'
        anc_split3.to_csv(f'{cs_name}_{anc_mon}3.csv', index=False)

        anc_split4 = anc_split[(cs_split.index >= 5001) & ((cs_split.index < 10000))].copy()
        anc_split4['ExternalID'] = anc_split['ExternalID']+'c'
        anc_split4.to_csv(f'{anc_name}_{anc_mon}4.csv', index=False)

total_anc = anc_template['itemLine_salesPrice'].sum()
print(f'{len(anc_template)} AN rows exported! The total sum amount is {total_anc}')    

# Filter for ancilliary rows, and create netsuite ancilliary template

cost_filter = df_heroes[((df_heroes['transaction-type'] == 'other-transaction')&
                   (df_heroes['amount-description'].isin(['FBAInboundTransportationProgramFee',
                                                          'FBAInboundTransportationFee',
                                                          'TaxDiscount',
                                                          'DisposalComplete',
                                                          'COMPENSATED_CLAWBACK',
                                                          'MISSING_FROM_INBOUND',
                                                          'GiftwrapChargeback',
                                                          'SalesTaxServiceFee',
                                                          'CouponRedemptionFee',
                                                          'Storage',
                                                          'Storage Fee',
                                                          'StorageRenewalBilling',
                                                          'MarketplaceFacilitatorTax-Other',
                                                          'MarketplaceFacilitatorTax-Principal',
                                                          'LowValueGoodsTax-Shipping',
                                                          'LowValueGoodsTax-Principal',
                                                          'MarketplaceFacilitatorTax-Shipping',
                                                          'Commingling VAT',
                                                          'WAREHOUSE_LOST',
                                                          'WAREHOUSE_LOST_MANUAL',
                                                          'Goodwill',
                                                          'Paid Services Fee',
                                                          'Current Reserve Amount',
                                                          'Previous Reserve Amount Balance',
                                                          'REVERSAL_REIMBURSEMENT',
                                                          'WAREHOUSE_DAMAGE',
                                                          'Warehouse damage'])))|                                    
                ((df_heroes['transaction-type'] == 'Refund')& 
                    (df_heroes['amount-description'].isin(['ShippingChargeback',
                                                           'RefundCommission',
                                                           'ShippingChargeback',
                                                           'TaxDiscount',
                                                           'MarketplaceFacilitatorTax-Principal',
                                                           'MarketplaceFacilitatorTax-Shipping',
                                                           'MarketplaceFacilitatorTax-RestockingFee',
                                                           'Commission',
                                                           'FREE_REPLACEMENT_REFUND_ITEMS'])))|
                ((df_heroes['transaction-type'] == 'Order')& 
                    (df_heroes['amount-description'].isin(['FBAPerUnitFulfillmentFee',
                                                           'Commission',
                                                           'MarketplaceFacilitatorVAT-Principal',
                                                           'MarketplaceFacilitatorVAT-Shipping',
                                                           'Service fee'])))|
                ((df_heroes['transaction-type'] == 'ServiceFee')& 
                    (df_heroes['amount-description'].isin(['TransactionTotalAmount'])))].copy()


costs = pd.read_csv('/home/markaw/heroes/netsuite_cashsales/cost_journal.csv', encoding='iso-8859-1')
costs_ns = cost_filter.merge(costs[['Costs','NS']], left_on='amount-description', right_on='Costs')
costs_grouped = costs_ns.groupby(['NS']).agg({'amount':'sum'})
costs_grouped.loc["COSTS TOTAL"] = costs_grouped.sum()
costs_grouped.loc["CASH SALES TOTAL"] = total_cs
costs_grouped.loc["ANCILLARY TOTAL"] = total_anc
costs_grouped.loc["FINAL REVENUE"] = costs_grouped.loc["COSTS TOTAL"]+costs_grouped.loc["CASH SALES TOTAL"]+costs_grouped.loc["ANCILLARY TOTAL"]
costs_grouped

In [101]:
cs_split2.head()

,ExternalID,tranId,customerRef,tranDate,postingPeriodRef,exchangeRate,locationRef,isTaxable,itemLine_itemRef,itemLine_quantity,itemLine_unitsRef,itemLine_salesPrice,itemLine_priceLevelRef,Subsiduary,Expected Deposit date
5006,CSTHE2306USb,,16 Amazon - US,21/05/2021,May 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,-0.90,Custom,2,15/06/2021
5007,CSTHE2306USb,,16 Amazon - US,26/05/2021,May 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,17.95,Custom,2,15/06/2021
5013,CSTHE2306USb,,16 Amazon - US,19/05/2021,May 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,17.95,Custom,2,15/06/2021
5016,CSTHE2306USb,,16 Amazon - US,19/05/2021,May 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,-0.90,Custom,2,15/06/2021
5017,CSTHE2306USb,,16 Amazon - US,24/05/2021,May 2021,1,Amazon FBA - KGP - US,TRUE,THF-FRD-WW-ZZ-10,1.0,Units,17.95,Custom,2,15/06/2021


In [100]:
result = (cs_split2['itemLine_salesPrice']*cs_split2['itemLine_quantity']).sum()
result

13837.0